In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support

import itertools


In [3]:
# file used to write preserve the results of the classfier
# confusion matrix and precision recall fscore matrix

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    """
    given a sklearn confusion matrix (cm), make a nice plot

    Arguments
    ---------
    cm:           confusion matrix from sklearn.metrics.confusion_matrix

    target_names: given classification classes such as [0, 1, 2]
                  the class names, for example: ['high', 'medium', 'low']

    title:        the text to display at the top of the matrix

    cmap:         the gradient of the values displayed from matplotlib.pyplot.cm
                  see http://matplotlib.org/examples/color/colormaps_reference.html
                  plt.get_cmap('jet') or plt.cm.Blues

    normalize:    If False, plot the raw numbers
                  If True, plot the proportions

    Usage
    -----
    plot_confusion_matrix(cm           = cm,                  # confusion matrix created by
                                                              # sklearn.metrics.confusion_matrix
                          normalize    = True,                # show proportions
                          target_names = y_labels_vals,       # list of names of the classes
                          title        = best_estimator_name) # title of graph

    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.tight_layout()
    return plt

In [4]:
##saving the classification report
def pandas_classification_report(y_true, y_pred):
    metrics_summary = precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred)
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    
    avg = list(precision_recall_fscore_support(
            y_true=y_true, 
            y_pred=y_pred,
            average='macro'))
    avg.append(accuracy_score(y_true, y_pred, normalize=True))
    metrics_sum_index = ['precision', 'recall', 'f1-score', 'support','accuracy']
    list_all=list(metrics_summary)
    list_all.append(cm.diagonal())
    class_report_df = pd.DataFrame(
        list_all,
        index=metrics_sum_index)

    support = class_report_df.loc['support']
    total = support.sum() 
    avg[-2] = total

    class_report_df['avg / total'] = avg

    return class_report_df.T

In [5]:
from commen_preprocess import *

.....start_cleaning.........
hashtag britain exit hashtag rape refugee


In [6]:
eng_train_dataset = pd.read_csv('../Data/english_dataset/english_dataset.tsv', sep='\t')
# #hindi_train_dataset = pd.read_csv('../Data/hindi_dataset/hindi_dataset.tsv', sep='\t',header=None)
# german_train_dataset = pd.read_csv('../Data/german_dataset/german_dataset_added_features.tsv', sep=',')
# eng_train_dataset=eng_train_dataset.drop(['Unnamed: 0'], axis=1)
# german_train_dataset=german_train_dataset.drop(['Unnamed: 0'], axis=1)

eng_train_dataset = eng_train_dataset.loc[eng_train_dataset['task_1'] == 'HOF']


In [7]:
eng_train_dataset.head()

,text_id,text,task_1,task_2,task_3
1,hasoc_en_2,@politico No. We should remember very clearly ...,HOF,HATE,TIN
7,hasoc_en_8,#ADOS #trendingnow #blacklivesmatter #justice ...,HOF,PRFN,TIN
11,hasoc_en_12,I don’t know how much more I can take! 45 is a...,HOF,HATE,TIN
15,hasoc_en_16,Good work @ICC keep going just destroy the who...,HOF,PRFN,TIN
23,hasoc_en_24,#ShameOnICC 1. ICC on Dhoni's gloves ...,HOF,HATE,TIN


In [8]:
l=eng_train_dataset['task_3'].value_counts()
print(l)

TIN    2041
UNT     220
Name: task_3, dtype: int64


In [9]:
import numpy as np
from tqdm import tqdm
import pickle
####loading laser embeddings for english dataset
def load_laser_embeddings():
        dim = 1024
        engX_commen = np.fromfile("../Data/english_dataset/embeddings_eng_task23_commen.raw", dtype=np.float32, count=-1)                                                                          
        engX_lib = np.fromfile("../Data/english_dataset/embeddings_eng_task23_lib.raw", dtype=np.float32, count=-1)                                                                          
        engX_commen.resize(engX_commen.shape[0] // dim, dim)                                                                          
        engX_lib.resize(engX_lib.shape[0] // dim, dim)                                                                          
        return engX_commen,engX_lib
    
def load_bert_embeddings():
        file = open('../Data/english_dataset/no_preprocess_bert_embed_task23.pkl', 'rb')
        embeds = pickle.load(file)
        return np.array(embeds)
        
def merge_feature(*args):
    feat_all=[]
    print(args[0].shape)
    for  i in tqdm(range(args[0].shape[0])):
        feat=[]
        for arg in args:
            feat+=list(arg[i])
        feat_all.append(feat)
    return feat_all


In [10]:
convert_label={
    'TIN':0,
    'UNT':1,
    
}


convert_reverse_label={
    0:'TIN',
    1:'UNT',
}


In [11]:
labels=eng_train_dataset['task_3'].values
engX_commen,engX_lib=load_laser_embeddings()
bert_embeds =load_bert_embeddings()

In [12]:
feat_all=merge_feature(engX_commen,engX_lib,bert_embeds)
len(feat_all[0])

 16%|█▌        | 359/2261 [00:00<00:00, 3579.74it/s]

(2261, 1024)


100%|██████████| 2261/2261 [00:00<00:00, 3672.58it/s]


2816

In [13]:
from sklearn.utils.multiclass import type_of_target

Classifier_Train_X=np.array(feat_all)
labels_int=[]
for i in range(len(labels)):
    labels_int.append(convert_label[labels[i]])

Classifier_Train_Y=np.array(labels_int,dtype='float64')
    

In [14]:
print(type_of_target(Classifier_Train_Y))
Classifier_Train_Y

binary


array([0., 0., 0., ..., 0., 1., 1.])

In [15]:
from sklearn.metrics import accuracy_score
import joblib
from sklearn.model_selection import StratifiedKFold as skf


###all classifier 
from catboost import CatBoostClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn import neighbors
from sklearn import ensemble
from sklearn import neural_network
from sklearn import linear_model
import lightgbm as lgbm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from nltk.classify.scikitlearn import SklearnClassifier


In [16]:
def train_model_no_ext(Classifier_Train_X,Classifier_Train_Y,model_type,save_model=False):
    kf = skf(n_splits=10,shuffle=True)
    y_total_preds=[] 
    y_total=[]
    count=0
    img_name = 'cm.png'
    report_name = 'report.csv'
    
    scale=list(Classifier_Train_Y).count(0)/list(Classifier_Train_Y).count(1)
    print(scale)
    
    if(save_model==True):
        Classifier=get_model(scale,m_type=model_type)
        Classifier.fit(Classifier_Train_X,Classifier_Train_Y)
        filename = model_type+'_eng_task_3.joblib.pkl'
        joblib.dump(Classifier, filename, compress=9)
#         filename1 = model_name+'select_features_eng_task1.joblib.pkl'
#         joblib.dump(model_featureSelection, filename1, compress=9)
    else:
        for train_index, test_index in kf.split(Classifier_Train_X,Classifier_Train_Y):
            X_train, X_test = Classifier_Train_X[train_index], Classifier_Train_X[test_index]
            y_train, y_test = Classifier_Train_Y[train_index], Classifier_Train_Y[test_index]

            classifier=get_model(scale,m_type=model_type)
            print(type(y_train))
            classifier.fit(X_train,y_train)
            y_preds = classifier.predict(X_test)
            for ele in y_test:
                y_total.append(ele)
            for ele in y_preds:
                y_total_preds.append(ele)
            y_pred_train = classifier.predict(X_train)
            print(y_pred_train)
            print(y_train)
            count=count+1       
            print('accuracy_train:',accuracy_score(y_train, y_pred_train),'accuracy_test:',accuracy_score(y_test, y_preds))
            print('TRAINING:')
            print(classification_report( y_train, y_pred_train ))
            print("TESTING:")
            print(classification_report( y_test, y_preds ))

        report = classification_report( y_total, y_total_preds )
        cm=confusion_matrix(y_total, y_total_preds)
        plt=plot_confusion_matrix(cm,normalize= True,target_names = ['TIN','UNT'],title = "Confusion Matrix")
        plt.savefig('eng_task3'+model_type+'_'+img_name)
        print(classifier)
        print(report)
        print(accuracy_score(y_total, y_total_preds))
        df_result=pandas_classification_report(y_total,y_total_preds)
        df_result.to_csv('eng_task3'+model_type+'_'+report_name,  sep=',')


In [20]:
def get_model(scale,m_type=None):
    if not m_type:
        print("ERROR: Please specify a model type!")
        return None
    if m_type == 'decision_tree_classifier':
        logreg = tree.DecisionTreeClassifier(max_features=1000,max_depth=3,class_weight='balanced')
    elif m_type == 'gaussian':
        logreg = GaussianNB()
    elif m_type == 'logistic_regression':
        logreg = LogisticRegression(n_jobs=10, random_state=42,class_weight='balanced',solver='liblinear')
    elif m_type == 'MLPClassifier':
#         logreg = neural_network.MLPClassifier((500))
        logreg = neural_network.MLPClassifier((100),random_state=42,early_stopping=True)
    elif m_type == 'RandomForestClassifier':
        logreg = ensemble.RandomForestClassifier(n_estimators=100, class_weight='balanced', n_jobs=12, max_depth=7)
    elif m_type == 'SVC':
        #logreg = LinearSVC(dual=False,max_iter=200)
        logreg = SVC(kernel='linear',random_state=1526)
    elif m_type == 'Catboost':
        logreg = CatBoostClassifier(iterations=100,learning_rate=0.2,
            l2_leaf_reg=500,depth=10,use_best_model=False, random_state=42,loss_function='MultiClass')
#         logreg = CatBoostClassifier(scale_pos_weight=0.8, random_seed=42,);
    elif m_type == 'XGB_classifier':
#         logreg=XGBClassifier(silent=False,eta=0.1,objective='binary:logistic',max_depth=5,min_child_weight=0,gamma=0.2,subsample=0.8, colsample_bytree = 0.8,scale_pos_weight=1,n_estimators=500,reg_lambda=3,nthread=12)
        logreg=XGBClassifier(silent=False,objective='multi:softmax',num_class=3,
                             reg_lambda=3,nthread=12, random_state=42)
    elif m_type == 'light_gbm':
        logreg = LGBMClassifier(objective='binary',max_depth=3,learning_rate=0.2,num_leaves=20,scale_pos_weight=scale,
                                boosting_type='gbdt', metric='binary_logloss',random_state=5,reg_lambda=20,silent=False)
    else:
        print("give correct model")
    print(logreg)
    return logreg

In [21]:
models_name=['decision_tree_classifier','gaussian','logistic_regression','MLPClassifier','RandomForestClassifier',
             'SVC','light_gbm']

In [22]:
for model in models_name:
    train_model_no_ext(Classifier_Train_X,Classifier_Train_Y,model)

9.277272727272727
DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=3,
            max_features=1000, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
<class 'numpy.ndarray'>
[0. 1. 0. ... 0. 1. 1.]
[0. 0. 0. ... 0. 1. 1.]
accuracy_train: 0.6622418879056047 accuracy_test: 0.5991189427312775
TRAINING:
              precision    recall  f1-score   support

         0.0       0.98      0.64      0.77      1836
         1.0       0.21      0.91      0.34       198

   micro avg       0.66      0.66      0.66      2034
   macro avg       0.60      0.77      0.56      2034
weighted avg       0.91      0.66      0.73      2034

TESTING:
              precision    recall  f1-score   support

         0.0       0.93      0.60      0.73       205
         1.0       0.14      0.59

KeyboardInterrupt: 

In [23]:
train_model_no_ext(Classifier_Train_X,Classifier_Train_Y,models_name[-1],save_model=True)

9.277272727272727
LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        importance_type='split', learning_rate=0.2, max_depth=3,
        metric='binary_logloss', min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=20, objective='binary', random_state=5,
        reg_alpha=0.0, reg_lambda=20, scale_pos_weight=9.277272727272727,
        silent=False, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)
